##### Install & Import Necessary Libraries

In [1]:
## Install the libraries

!pip install sqlalchemy beautifulsoup4 selenium requests

   ---------------------------------------- 0.0/9.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/9.7 MB 1.4 MB/s eta 0:00:08
   ---------------------------------------- 0.0/9.7 MB 1.4 MB/s eta 0:00:08
   ---------------------------------------- 0.0/9.7 MB 281.8 kB/s eta 0:00:35
   ---------------------------------------- 0.1/9.7 MB 459.5 kB/s eta 0:00:21
   ---------------------------------------- 0.1/9.7 MB 504.4 kB/s eta 0:00:19
    --------------------------------------- 0.2/9.7 MB 579.6 kB/s eta 0:00:17
    --------------------------------------- 0.2/9.7 MB 724.0 kB/s eta 0:00:14
   - -------------------------------------- 0.2/9.7 MB 718.0 kB/s eta 0:00:14
   - -------------------------------------- 0.3/9.7 MB 706.2 kB/s eta 0:00:14
   - -------------------------------------- 0.3/9.7 MB 703.0 kB/s eta 0:00:14
   - -------------------------------------- 0.4/9.7 MB 716.8 kB/s eta 0:00:14
   - -------------------------------------- 0.4/9.7 MB 730.1 kB/s eta 0:00:13

In [26]:
## Import the libraries
import requests as rq, pandas as pd, re, time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.edge.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.edge.options import Options
from sqlalchemy import create_engine
import numpy as np
import os
from dotenv import load_dotenv

In [4]:
## Testing url
url = 'https://www.aliexpress.com/w/wholesale-generators-portable.html?spm=a2g0o.productlist.auto_suggest.1.7e84UF2TUF2T64rq.get(url)'
response = rq.get(url)
response.status_code

200

In [3]:
## Configure Selenium ChromeDriver options
options = Options()
options.use_chromium = True
service = Service(executable_path=r'Driver_Notes\msedgedriver.exe')

## Initialize the webdriver
driver = webdriver.Edge(service=service, options=options)

# Define the URL
# url = 'https://www.aliexpress.com/w/wholesale-generators-portable.html?spm=a2g0o.productlist.auto_suggest.1.7e84UF2TUF2T64rq.get(url)'

# List to store extracted values
product_names = []
stores = []
prices = []
shipping_fees = []
extra_discounts = []
original_prices = []
ratings = []
qty_solds = []

total_page_no = 10

for page in range(1, total_page_no+1):
    url = f'https://www.aliexpress.com/w/wholesale-generators-portable.html?page={page}&g=y&SearchText=generators+portable'
    driver.get(url)
    time.sleep(60)

    # Use of BeautifulSoup
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # Adjust the class selectors based on the current website structure
    generators = soup.find_all('div', class_ = 'list--gallery--C2f2tvm search-item-card-wrapper-gallery')

    for genn in generators:
        # product_name
        product_name = genn.find('div', class_ = 'multi--title--G7dOCj3').text

        # stores
        try:
            store = genn.find('a', class_ = 'cards--storeLink--XkKUQFS').text
        except AttributeError:
            store = ''

        # prices
        price = genn.find('div', class_ = 'multi--price-sale--U-S0jtj').text.replace('NGN', '').replace(',', '')

        # extra discount
        try:
            extra_discount = genn.find('span', class_ = 'tag--text--1BSEXVh tag--textStyle--3dc7wLU multi--superStyle--1jUmObG').text
        except AttributeError:
            extra_discount = ''
        
        # original price
        try:
            original_price = genn.find('div', class_ = 'multi--price-original--1zEQqOK').text
        except AttributeError:
            original_price = ''
        
        # Quantity sold
        try:
            qty_solds_text = genn.find('span', class_ = 'multi--trade--Ktbl2jB').text
            qty_sold = qty_solds_text.split()[0] if qty_solds_text else ''
        except AttributeError:
            qty_sold = ''

        # Ratings
        try:
            rating = genn.find('div', class_ = 'multi--starList--t9_CAj2').text
        except AttributeError:
            rating = ''

        # Append
        product_names.append(product_name)
        stores.append(store)
        prices.append(price)
        extra_discounts.append(extra_discount)
        original_prices.append(original_price)
        ratings.append(rating)
        qty_solds.append(qty_sold)

# Close the driver
driver.quit()

# Create the dataframe
data = {
    'product_name' : product_names,
    'stores' : stores,
    'prices' : prices,
    'extra_discounts' : extra_discounts,
    'original_prices' : original_prices,
    'ratings' : ratings,
    'qty_solds' : qty_solds
}

jumba_df = pd.DataFrame(data)

In [5]:
## Display the dataframe
jumba_df.head(20)

,product_name,stores,prices,extra_discounts,original_prices,ratings,qty_solds
0,200W Portable Power Station 220V LiFePO4 Batte...,Wholesale Store Store,17088.69,,,,6
1,200W Portable Power Station 220V Power Banks 6...,Wholesale Main Store Store,14590.66,Extra 2% off with coins,,,3
2,220V 60800mah Portable Power Station Camping B...,Wholesale Dealership Store,15572.49,,,,
3,Зарядные Станции 3000W 2000W 1000W 600W Portab...,Portable Power Station Store,891470.21,,,,1
4,120W 220V Portable Power Station 58000mAh Outd...,Shop1104193228 Store,19347.17,,,,
5,Portable 5.5V Hand Generator USB ABS Outdoor E...,New House Global Store,6872.85,,"NGN6,872.85",,32
6,4 Portable Hand Crank Charger Mini Dynamo Gene...,Preferred Daily tools Store,2197.94,Extra 2% off with coins,,,4
7,10KW 20KW Low Speed Permanent Magnet Generator...,Meteor Energy Store,985344.78,Extra 3% off with coins,,,47
8,2400W Portable power station Lithium Battery A...,Fishing Gear Outdoor Sports Store,6002168.57,Extra 5% off with coins,,,1
9,1pc USB Hand Crank Charger Generator Manual Dy...,Tools Top Store,1800.47,,,,8


In [4]:
## Display the dataframe
jumba_df.shape

(100, 7)

##### Transformation

In [8]:
# Products table
product_columns = ['product_name', 'stores', 'prices']
product_table = jumba_df[product_columns].copy()

# Creates product ID column
product_table['product_id'] = range(1, len(product_table)+1)
product_table = product_table[['product_id','product_name', 'stores', 'prices']]

In [9]:
product_table.head()

,product_id,product_name,stores,prices
0,1,200W Portable Power Station 220V LiFePO4 Batte...,Wholesale Store Store,17088.69
1,2,200W Portable Power Station 220V Power Banks 6...,Wholesale Main Store Store,14590.66
2,3,220V 60800mah Portable Power Station Camping B...,Wholesale Dealership Store,15572.49
3,4,Зарядные Станции 3000W 2000W 1000W 600W Portab...,Portable Power Station Store,891470.21
4,5,120W 220V Portable Power Station 58000mAh Outd...,Shop1104193228 Store,19347.17


In [19]:
# Discount table
discount_columns = ['product_name', 'prices', 'original_prices', 'extra_discounts']
discount_table = jumba_df[discount_columns].copy()

# Remove 'NGN' and ',' from original prices and convert it to float. We're also addressing empty rows, all with lambda
discount_table['original_prices'] = discount_table['original_prices'].apply(lambda x: np.nan if x.strip() == '' else x.replace('NGN','').replace(',','')).astype(float)

# Extract discount value as integer
discount_table['extra_discounts'] = discount_table['extra_discounts'].str.extract(r'(\d+)%').astype(float)

# Create the discount ID column
discount_table['discount_id'] = range(1, len(discount_table)+1)
discount_table = discount_table[['discount_id','product_name', 'prices', 'original_prices', 'extra_discounts']]

In [20]:
discount_table.head()

,discount_id,product_name,prices,original_prices,extra_discounts
0,1,200W Portable Power Station 220V LiFePO4 Batte...,17088.69,NaN,NaN
1,2,200W Portable Power Station 220V Power Banks 6...,14590.66,NaN,2.0
2,3,220V 60800mah Portable Power Station Camping B...,15572.49,NaN,NaN
3,4,Зарядные Станции 3000W 2000W 1000W 600W Portab...,891470.21,NaN,NaN
4,5,120W 220V Portable Power Station 58000mAh Outd...,19347.17,NaN,NaN


In [21]:
# Sales table
sales_columns = ['product_name', 'prices', 'qty_solds']
sales_table = jumba_df[sales_columns].copy()

# Creates sales ID column
sales_table['sales_id'] = range(1, len(sales_table)+1)
sales_table = sales_table[['sales_id','product_name', 'prices', 'qty_solds']]

In [22]:
sales_table.head()

,sales_id,product_name,prices,qty_solds
0,1,200W Portable Power Station 220V LiFePO4 Batte...,17088.69,6
1,2,200W Portable Power Station 220V Power Banks 6...,14590.66,3
2,3,220V 60800mah Portable Power Station Camping B...,15572.49,
3,4,Зарядные Станции 3000W 2000W 1000W 600W Portab...,891470.21,1
4,5,120W 220V Portable Power Station 58000mAh Outd...,19347.17,


In [ ]:
# To Do
## Add shipping table - product_name, shipping_price, shipping_free_status

In [25]:
## Saving to csv
product_table.to_csv('product_table.csv', index=False)
sales_table.to_csv('sales_table.csv', index=False)
discount_table.to_csv('discount_table.csv', index=False)

##### Loading to DB

In [34]:
load_dotenv(override=True)

True

In [35]:
# define DB Params
# Credentials
hostname = os.getenv('hostname')
username = os.getenv('username')
password = os.getenv('password')
port = os.getenv('port')
db_name = os.getenv('db_name')

# Create the DB engine
db_url = f'postgresql+psycopg2://{username}:{password}@{hostname}:{port}/{db_name}'
engine = create_engine(db_url)

In [36]:
with engine.connect() as connection:
    product_table.to_sql('products', con=connection, schema="EDW", index=False, if_exists='replace')
    sales_table.to_sql('sales', con=connection, schema="EDW", index=False, if_exists='replace')
    discount_table.to_sql('discount', con=connection, schema="EDW", index=False, if_exists='replace')

    print('all tables loaded successfully')

all tables loaded successfully
